In [1]:
!pip install ortools

  Obtaining dependency information for ortools from https://files.pythonhosted.org/packages/a5/3b/6cc4f57727db5efc7a5faaafcecb257cbd2b81d1d4b344dd895ddadbf2b0/ortools-9.7.2996-cp310-cp310-win_amd64.whl.metadata
  Obtaining dependency information for protobuf>=4.23.3 from https://files.pythonhosted.org/packages/1c/1f/e64ec8bc540266178d4d0e0255e0c1427ba284ba951e193d193a238b09bb/protobuf-4.24.0-cp310-abi3-win_amd64.whl.metadata
   ---------------------------------------- 0.0/44.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/44.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/44.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/44.9 MB 262.6 kB/s eta 0:02:51
   ---------------------------------------- 0.0/44.9 MB 245.8 kB/s eta 0:03:03
   ---------------------------------------- 0.1/44.9 MB 708.1 kB/s eta 0:01:04
   ---------------------------------------- 0.5/44.9 MB 1.9 MB/s eta 0:00:24
    --------------------------------------- 0.8

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorboard 2.11.0 requires werkzeug>=1.0.1, which is not installed.
tensorboard 2.11.0 requires protobuf<4,>=3.9.2, but you have protobuf 4.24.0 which is incompatible.
tensorflow-intel 2.11.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.24.0 which is incompatible.



Simple Travelling Salesman Problem.

A description of the problem can be found here:
http://en.wikipedia.org/wiki/Travelling_salesperson_problem.



In [2]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp


def create_data_model():
    """Stores the data for the problem."""
    data = {}
    # Locations in block units
    locations = [
        # fmt:off
      (4, 4),  # depot
      (2, 0), (8, 0),  # locations to visit
      (0, 1), (1, 1),
      (5, 2), (7, 2),
      (3, 3), (6, 3),
      (5, 5), (8, 5),
      (1, 6), (2, 6),
      (3, 7), (6, 7),
      (0, 8), (7, 8)
        # fmt:on
    ]
    # Convert locations in meters using a city block dimension of 114m x 80m.
    data["locations"] = [(l[0] * 114, l[1] * 80) for l in locations]
    data["num_vehicles"] = 1
    data["depot"] = 0
    return data


def create_distance_callback(data, manager):
    """Creates callback to return distance between points."""
    distances_ = {}
    index_manager_ = manager
    # precompute distance between location to have distance callback in O(1)
    for from_counter, from_node in enumerate(data["locations"]):
        distances_[from_counter] = {}
        for to_counter, to_node in enumerate(data["locations"]):
            if from_counter == to_counter:
                distances_[from_counter][to_counter] = 0
            else:
                distances_[from_counter][to_counter] = abs(
                    from_node[0] - to_node[0]
                ) + abs(from_node[1] - to_node[1])

    def distance_callback(from_index, to_index):
        """Returns the manhattan distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = index_manager_.IndexToNode(from_index)
        to_node = index_manager_.IndexToNode(to_index)
        return distances_[from_node][to_node]

    return distance_callback


def print_solution(manager, routing, assignment):
    """Prints assignment on console."""
    print(f"Objective: {assignment.ObjectiveValue()}")
    index = routing.Start(0)
    plan_output = "Route for vehicle 0:\n"
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += f" {manager.IndexToNode(index)} ->"
        previous_index = index
        index = assignment.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += f" {manager.IndexToNode(index)}\n"
    plan_output += f"Distance of the route: {route_distance}m\n"
    print(plan_output)


def main():
    """Entry point of the program."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(data["locations"]), data["num_vehicles"], data["depot"]
    )

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    distance_callback = create_distance_callback(data, manager)
    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )

    # Solve the problem.
    assignment = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if assignment:
        print_solution(manager, routing, assignment)


main()



Objective: 4384
Route for vehicle 0:
 0 -> 9 -> 5 -> 8 -> 6 -> 2 -> 10 -> 16 -> 14 -> 13 -> 12 -> 11 -> 15 -> 3 -> 4 -> 1 -> 7 -> 0
Distance of the route: 4384m

